https://github.com/janivanecky/Artistic-Style


In [1]:
import argparse
import numpy as np
import tensorflow as tf
import vgg
import style
from matplotlib.pyplot import imshow
from PIL import Image

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
args_crop='left'
args_shape=None
args_scale=1.0
args_style_weight = 0.5

In [4]:
content_image = style.load_image('me.jpeg', args_crop, args_shape, args_scale)
WIDTH, HEIGHT = content_image.shape[1], content_image.shape[0]
content_image = content_image.reshape((1, HEIGHT, WIDTH, 3))
style_image = style.load_image('uae.jpg', args_crop=None, shape=(WIDTH, HEIGHT))
style_image = style_image.reshape((1, HEIGHT, WIDTH, 3))

In [5]:
print('prepare networks')
images = np.concatenate((content_image, style_image), 0).astype(np.float32)
constants = tf.constant(images)
with tf.name_scope("constant"):
    vgg_const = vgg.Vgg19()
    vgg_const.build(constants)

# use noise as an initial image 
#input_image = tf.Variable(tf.truncated_normal([1, HEIGHT, WIDTH, 3], 0.5, 0.1))
# use content image as an initial image
input_image = tf.Variable(np.expand_dims(images[0,:,:,:], 0))
with tf.name_scope("variable"):
    vgg_var = vgg.Vgg19()
    vgg_var.build(input_image)

# which layers we want to use?
# style_layers_const = [vgg_const.conv1_1, vgg_const.conv2_1, vgg_const.conv3_1, vgg_const.conv4_1, vgg_const.conv5_1]
# style_layers_var = [vgg_var.conv1_1, vgg_var.conv2_1, vgg_var.conv3_1, vgg_var.conv4_1, vgg_var.conv5_1]
# content_layer_const = vgg_const.conv4_2
# content_layer_var = vgg_var.conv4_2
style_layers_const = [vgg_const.conv1_1, vgg_const.conv2_1, vgg_const.conv3_1, vgg_const.conv4_1, vgg_const.conv5_1]
style_layers_var = [vgg_var.conv1_1, vgg_var.conv2_1, vgg_var.conv3_1, vgg_var.conv4_1, vgg_var.conv5_1]
content_layer_const = vgg_const.conv4_4
content_layer_var = vgg_var.conv4_4

# get activations of content and style images as TF constants
print('calc layers')
sess = tf.Session()
layers = sess.run([content_layer_const] + style_layers_const)
content_layer_const = tf.constant(np.expand_dims(layers[0][0,:,:,:], 0))
style_layers_const = [tf.constant(np.expand_dims(layer[1,:,:,:], 0)) for layer in layers[1:]]

print('compose the loss function')
content_style_ratio = 1e-4
loss_content = style.content_loss(content_layer_const, content_layer_var)
loss_style = style.style_loss(style_layers_const, style_layers_var)
style_weight = float(args_style_weight)
overall_loss = (1 - style_weight) * content_style_ratio * loss_content + style_weight * loss_style

print('set up optimizer ')
output_image = tf.clip_by_value(tf.squeeze(input_image, [0]), 0, 1)
train = tf.train.AdamOptimizer(learning_rate=0.05).minimize(overall_loss)

print('training:')
init = tf.initialize_all_variables()
sess.run(init)
min_loss, best_image = float("inf"), None
for i in xrange(0, 700):
    image, loss, _ = sess.run([output_image, overall_loss, train])
    if i % 25 == 0:
        print 'Iteration {}: {}'.format(i, loss)
        if(loss < min_loss):
            min_loss, best_image = loss, image

# save the result
best_image = np.clip(best_image, 0, 1)
best_image = np.reshape(best_image, (HEIGHT,WIDTH,3))
result = Image.fromarray(np.uint8(best_image * 255))
#result.save(args.output_image)
result.show()
print('done', 'min loss=', min_loss)

prepare networks


IOError: [Errno 2] No such file or directory: '/Users/spastukhov/Documents/ds/studying-ml/Art Style Transfer/vgg19.npy'